In [2]:
import jax
import sys
import os
import numpy as np
import pytest
import jax.numpy as jnp
from matplotlib import pyplot as plt 

In [3]:
import logging
import copy
from pathlib import Path

current_notebook_folder = Path().resolve()
target_file_path = current_notebook_folder.parent / 'spring_run_test.py'
absolute_target_path = target_file_path.resolve()
sys.path.append(str(current_notebook_folder.parent))
from spring_run_test import run_molecule,plot_losses
#The running function is in spring_run_test.py
from ml_collections import config_dict
import vmcnet.train as train

In [ ]:
config = train.default_config.get_default_config()
#config = config_dict.ConfigDict()
config.save_to_current_datetime_subfolder=False 
#config.subfolder_name=C_preliminary 
config.problem.ion_pos=((0.0, 0.0, -2.0), (0.0, 0.0, 2.0))
config.problem.ion_charges=(7.0, 7.0) 
config.problem.nelec=(7, 7)
# N2 with bond distance 4.0 Bohr
config.vmc.optimizer_type='fisher_acc' # use the ANGD-Fisher-Rao method
config.vmc.nchains=2000   
config.vmc.optimizer.fisher_acc.nchains_train=config.vmc.nchains  #train
config.vmc.nepochs=30000
config.vmc.nburns=10000
config.eval.nepochs=0 
config.model.ferminet.ndeterminants=16 
config.model.ferminet.full_det=True 
config.vmc.checkpoint_every=1000
config.vmc.nsteps_per_param_update=10    
#config.distribute=True
import math

lr=math.sqrt(0.001)
# lr1, lr2 are step sizes (learning rates) for the parameter and the cotagent variable Phi, respectively. 
# We set the same values for all experiments.
lr1=lr
lr2=lr
lrd=5e-5  #linear learning rate decay
config.vmc.optimizer.fisher_acc.learning_rate1=lr1
config.vmc.optimizer.fisher_acc.schedule_type1 = "inverse_time"
config.vmc.optimizer.fisher_acc.learning_decay_rate1=lrd


config.vmc.optimizer.fisher_acc.learning_rate2=lr2
config.vmc.optimizer.fisher_acc.schedule_type2 = "inverse_time"  
config.vmc.optimizer.fisher_acc.learning_decay_rate2=lrd
config.vmc.optimizer.fisher_acc.learning_rate2_upper=0.5


config.vmc.optimizer.fisher_acc.alpha=0.5/lr2   # initial alpha
config.vmc.optimizer.fisher_acc.balance=lr1/lr2 # a hyper-parameter only for tests
config.vmc.optimizer.fisher_acc.alpha_schedule_type="inverse_time"  
config.vmc.optimizer.fisher_acc.alpha_decay_rate=1e-4  #linear alpha decay
config.vmc.optimizer.fisher_acc.alpha_lower_bound=0.1/lr2 #min alpha value


config.vmc.optimizer.fisher_acc.beta=0.05 # initial beta
config.vmc.optimizer.fisher_acc.beta_schedule_type="inverse_time" 
config.vmc.optimizer.fisher_acc.beta_decay_rate=5e-5 #linear beta decay

config.vmc.optimizer.fisher_acc.mu=0.99  #decay for projected momentum used in spring

config.vmc.optimizer.fisher_acc.constrain_norm=True
config.vmc.optimizer.fisher_acc.norm_constraint=5.0 # restart threshold for Phi
config.vmc.optimizer.fisher_acc.prev_eta_proj='Null_O' #using projected momentum (spring)
config.vmc.optimizer.fisher_acc.restart=False
config.vmc.optimizer.fisher_acc.nchains=config.vmc.nchains
config.vmc.optimizer.fisher_acc.damping=0.005* config.vmc.optimizer.fisher_acc.nchains #multiplying the batch size for scaling 

config.vmc.print_epoch=1000 #print the iteration number every 1000 iterations
config.params_init=None

config.model = train.default_config.choose_model_type_in_model_config(config.model)

In [ ]:
# A running example.
# "Warning: Gradient norm exceeds the constraint" means the restart step.
# loss_list, var_list contain the loss and variance values v.s. iterations respectively.
params, optimizer_state, data, key, nans_detected,loss_list,var_list,metric_list=run_molecule(config)

pmap False
epoch: 0
